# Introduction à l'Optimisation de Chaîne d'Approvisionnement

##  Objectif

Ce projet démontre comment optimiser les paramètres d'inventaire d'une chaîne d'approvisionnement simple en utilisant différentes techniques d'optimisation basées sur la simulation.

## Cas d'étude simplifié

- **1 Fabricant** (Samsung) : Capacité illimitée
- **1 Grossiste** (LDLC) : Stock limité avec politique (S,s)
- **1 Détaillant** (Fnac) : Stock limité avec politique (S,s)
- **Clients** : Arrivées selon processus de Poisson

## Variables de décision
- `S_grossiste` : Capacité maximale de stock du grossiste
- `s_grossiste` : Seuil de réapprovisionnement du grossiste
- `S_detaillant` : Capacité maximale de stock du détaillant
- `s_detaillant` : Seuil de réapprovisionnement du détaillant

In [ ]:
# %%
# Imports nécessaires
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import simpy
from typing import Dict, List, Tuple, Optional
import time
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("✅ Imports réussis!")

✅ Imports réussis!


# 🔧 Configuration des paramètres

In [ ]:
# Paramètres économiques et logistiques
CONFIG = {
    # Paramètres économiques
    'marge_smartphone': 80,      # Profit par smartphone vendu (€)
    'cout_stockage_grossiste': 2.0,   # Coût de stockage par unité/jour (€)
    'cout_stockage_detaillant': 5.0,  # Coût de stockage par unité/jour (€)
    
    # Paramètres logistiques
    'cout_livraison_fabricant': 800,   # Coût de livraison fabricant->grossiste (€)
    'delai_livraison_fabricant': 5,    # Délai de livraison (jours)
    'cout_livraison_grossiste': 50,    # Coût de livraison grossiste->détaillant (€)
    'delai_livraison_grossiste': 1,    # Délai de livraison (jours)
    
    # Paramètres de demande
    'taux_arrivee_clients': 15,        # Clients par jour (λ pour Poisson)
    'achat_min': 1,                    # Minimum de smartphones par client
    'achat_max': 3,                    # Maximum de smartphones par client
    
    # Paramètres de simulation
    'duree_simulation': 90,            # Durée de simulation (jours)
    'nombre_replications': 50,         # Nombre de réplications pour moyennes 
}

# Espaces de recherche pour l'optimisation
BOUNDS = {
    'S_grossiste': (600, 1200),   # Capacité stock grossiste 
    's_grossiste': (200, 500),    # Seuil réappro grossiste
    'S_detaillant': (100, 300),   # Capacité stock détaillant
    's_detaillant': (30, 100),    # Seuil réappro détaillant
}

print("📊 Configuration chargée:")
for key, value in CONFIG.items():
    print(f"  - {key}: {value}")

📊 Configuration chargée:
  - marge_smartphone: 80
  - cout_stockage_grossiste: 2.0
  - cout_stockage_detaillant: 5.0
  - cout_livraison_fabricant: 800
  - delai_livraison_fabricant: 5
  - cout_livraison_grossiste: 50
  - delai_livraison_grossiste: 1
  - taux_arrivee_clients: 15
  - achat_min: 1
  - achat_max: 3
  - duree_simulation: 90
  - nombre_replications: 50


 #  Architecture de la solution
 
 ## Approche méthodologique
 
 1. **Modélisation** : Simulation événementielle avec SimPy
 2. **Exploration** : Analyse de l'espace de conception
 3. **Optimisation** : Comparaison de plusieurs algorithmes
    - Monte Carlo (recherche aléatoire)
    - Recherche par grille
    - Métamodèle GPR + Optimisation
    - Algorithmes évolutionnaires
 4. **Analyse** : Comparaison des performances

In [3]:
# Fonction utilitaire pour sauvegarder les résultats
import os

def create_directories():
    """Créer les répertoires nécessaires"""
    dirs = ['results', 'results/figures', 'results/data', 'src']
    for d in dirs:
        os.makedirs(d, exist_ok=True)
    print("✅ Répertoires créés")

create_directories()

✅ Répertoires créés


 # Métriques de performance
 
 ## Métriques principales
 
 1. **Profit net quotidien** = Revenus - Coûts totaux
    - Revenus = Nombre de ventes × Marge unitaire
    - Coûts = Coûts de stockage + Coûts de livraison
 
 2. **Taux de service** = Clients servis / Total clients × 100%
 
 3. **Niveau de stock moyen** = Moyenne temporelle du stock
 
 4. **Temps de rupture** = % du temps avec stock = 0


In [ ]:
# Classes pour stocker les résultats
from dataclasses import dataclass
from typing import List

@dataclass
class SimulationResults:
    """Résultats d'une simulation"""
    profit_net: float
    taux_service: float
    stock_moyen_grossiste: float
    stock_moyen_detaillant: float
    cout_stockage_total: float
    cout_livraison_total: float
    ventes_totales: int
    clients_perdus: int
    
    def __str__(self):
        return f"""
        📊 Résultats de simulation:
        - Profit net: {self.profit_net:.2f}€/jour
        - Taux de service: {self.taux_service:.1%}
        - Stock moyen (G/D): {self.stock_moyen_grossiste:.0f}/{self.stock_moyen_detaillant:.0f}
        - Ventes totales: {self.ventes_totales}
        """

@dataclass
class OptimizationResults:
    """Résultats d'optimisation"""
    algorithm: str
    best_params: Dict[str, float]
    best_profit: float
    convergence_history: List[float]
    computation_time: float
    n_evaluations: int
    
    def summary(self):
        return pd.DataFrame({
            'Algorithme': [self.algorithm],
            'Profit optimal': [f"{self.best_profit:.2f}€"],
            'Temps calcul': [f"{self.computation_time:.1f}s"],
            'Évaluations': [self.n_evaluations],
            'S_grossiste': [self.best_params['S_grossiste']],
            's_grossiste': [self.best_params['s_grossiste']],
            'S_detaillant': [self.best_params['S_detaillant']],
            's_detaillant': [self.best_params['s_detaillant']]
        })

print("✅ Classes de résultats définies")

 
 Dans le notebook suivant, nous allons implémenter le modèle de simulation SimPy simplifié.
 
 ## Points clés à retenir:
 
 1. **Simplification** : 1 seul détaillant pour faciliter l'analyse
 2. **Politique (S,s)** : Commande quand stock < s, jusqu'à atteindre S
 3. **Objectif** : Maximiser le profit net quotidien
 4. **Contraintes** : Satisfaire la demande client tout en minimisant les coûts


In [4]:
import pickle

with open('results/data/config.pkl', 'wb') as f:
    pickle.dump({'CONFIG': CONFIG, 'BOUNDS': BOUNDS}, f)
    
print("💾 Configuration sauvegardée dans results/data/config.pkl")
print("\n➡️ Passez au notebook 02_simulation_base.ipynb")

💾 Configuration sauvegardée dans results/data/config.pkl

➡️ Passez au notebook 02_simulation_base.ipynb
